In [152]:
import pandas as pd
import numpy as np
import random
import gc
from os import listdir
from xgboost import XGBClassifier
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.optimizers import SGD, RMSprop
import pickle
from sklearn.externals import joblib
from sklearn import tree
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split, cross_val_score
from mlxtend.classifier import EnsembleVoteClassifier

# Note types
- 0: nothing
- 1: step
- 2: hold start
- 3: hold/roll end
- 4: roll start
- M: mine

# Classes
- 0: nothing
- 1: one note
- 2: two notes
- 3: three or four notes
- 4: hold start
- 5: roll start
- 6: mine

In [164]:
samples_back_included = 8
num_classes = 7
num_features = 40
num_features_total = (num_features * samples_back_included) + 4
save_files = listdir('data')

def get_features_for_index(beat_features, index):
    return beat_features[index] if index => 0 else [0] * num_features

def get_class_for_index(notes, index):
    if index < 0:
        return 0
    row = notes[index][0]
    (steps, holds, rolls, mines) = [row.count(char) for char in ['1', '2', '4', 'M']]
    steps += (holds + rolls)
    return np.argmax([int(i) for i in [steps == 0 and mines == 0, steps == 1, steps == 2, steps > 2, holds > 0, rolls > 0, mines > 0]])
    
importance_rankings = [48, 24, 12, 16, 6, 8, 3, 4, 2, 1]
def get_beat_importance(index):
    for i in range(len(importance_rankings)):
        if index % importance_rankings[i] == 0:
            return i

def get_features_for_song(X, y, key):
    if '{0}_beat_features.csv'.format(key) in save_files and '{0}_notes.csv'.format(key) in save_files:
        beat_features_rotated = pd.read_csv('data/{0}_beat_features.csv'.format(key)).values
        notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        beat_features = np.flipud(np.rot90(np.array(beat_features_rotated)))
        num_notes = min(len(notes), len(beat_features))
        for i in range(num_notes):
            features = [feature for j in range(samples_back_included) for feature in get_features_for_index(beat_features, i - j)]
            features.extend([i % 48, get_beat_importance(i), i / 48, num_notes - i / 48])
            X.append(features)
            y.append(get_class_for_index(notes, i))

# Total 243 songs
songs_to_use = pd.read_csv('data/songs_to_use.csv').values
np.random.shuffle(songs_to_use)
def build_training_data(start, end):
    X = []
    y = []
    for song_data in songs_to_use[start:end]:
        get_features_for_song(X, y, song_data[0])
    return np.array(X), np.array(y)

In [166]:
X_train, y_train = build_training_data(0, 194)
X_test, y_test = build_training_data(194, 243)

In [167]:
gc.collect()

14

## Final Models

#### max_depth = 3 (0.881996974281)
0.885299268107
CPU times: user 1h 55min 59s, sys: 33.4 s, total: 1h 56min 33s
Wall time: 31min 36s

#### max_depth = 5 (0.882895702581)
0.887660335563
CPU times: user 3h 8min 31s, sys: 35.2 s, total: 3h 9min 6s
Wall time: 51min 12s

#### max_depth=7 (0.883434939561)
0.896775328155
CPU times: user 4h 20min 18s, sys: 52 s, total: 4h 21min 10s
Wall time: 1h 10min 44s

#### max_depth=10 (0.882386423211)
0.916481448221
CPU times: user 6h 5min 3s, sys: 1min 37s, total: 6h 6min 40s
Wall time: 1h 40min 41s

In [ ]:
xgb_clf = XGBClassifier(max_depth=7, min_child_weight=8, learning_rate=0.05, seed=0, n_estimators=100, subsample=0.80, colsample_bytree=0.80, objective="multi:softprob")
# 0.896775328155
# 0.883434939561

In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 100, max_features=200, min_samples_leaf=12, verbose=True)
# 0.942960202356
# 0.882506253651

In [ ]:
sgd_clf = SGDClassifier(loss="modified_huber", n_iter=50)
# 0.815508609316
# 0.827428696147

In [223]:
eclf = EnsembleVoteClassifier(clfs=[xgb_clf, rf_clf, sgd_clf], weights=[1, 1, 1], voting='soft')
#eclf.fit(X_train, y_train)
print (eclf.score(X_train, y_train))
print (eclf.score(X_test, y_test))
joblib.dump(eclf, 'models/song_class_eclf/clf.pkl')
gc.collect()

AttributeError: 'EnsembleVoteClassifier' object has no attribute 'clfs_'

## Not Using

In [ ]:
%%time
model = Sequential()

model.add(Dense(512, input_shape=(324,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

model.fit(X_train, y_train, nb_epoch=50, batch_size=64, verbose=1)
print (model.evaluate(X_test, y_test, batch_size=64))
model.save('models/song_class_model.h5')
model = None
gc.collect()
# 0.884
# 0.86375278980600079